In [ ]:
#First, we import all the necessary library.This neural network is implemented in keras.
#Next we will import all the module that we'll use through out this notebook
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from skimage.color import gray2rgb,rgb2gray,label2rgb
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
import tensorflow as tf

img_size=224

In [ ]:
directory=['dyed-lifted-polyps','dyed-resection-margins','esophagitis','normal-cecum','normal-pylorus','normal-z-line','polyps','ulcerative-colitis']
y=[]
for i in directory:
    print(i)
    y.append(len(os.listdir(os.path.join('../input/dataset/kvasir-dataset-v2' +'/' +i))))
print(y)                  

In [ ]:
!pip install split-folders
input_folder = '../input/dataset/kvasir-dataset-v2'
import splitfolders

splitfolders.ratio(input_folder,output='data2',seed=42,ratio=(.9,.1),group_prefix=None)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=30,
    data_format='channels_last',
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    validation_split=0.112
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
'./data2/train',
target_size=(img_size,img_size),
batch_size=16,
color_mode = 'rgb',
shuffle = True,
seed = 42,
class_mode='categorical',
subset = 'training'
)


valid_generator = train_datagen.flow_from_directory(
directory="./data2/train",
target_size=(img_size, img_size),
color_mode="rgb",
batch_size=16,
class_mode="categorical",
shuffle=True,
seed=42,
subset ='validation'

)


In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
 )

test_generator = test_datagen.flow_from_directory(
    directory="./data2/val",
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=1,
    class_mode='binary',
    shuffle=False,
    seed=42
    )

In [ ]:
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Conv3D,Conv2D,MaxPooling2D,Input,Dense,Flatten,Dropout,Activation,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

base_model=tf.keras.applications.DenseNet201(
    input_shape=(img_size, img_size, 3),
    include_top=False)
base_model.summary()


In [ ]:
#  only training some layers 

for layers in base_model.layers[:-11]:
    layers.trainable=False
  
for layers in base_model.layers:
    print(layers ,layers.trainable)

In [ ]:
def createmodel():
    model = Sequential()
    model.add(base_model) #loads above imported pretrained model
    # # here we added new layers
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(8, activation='softmax', name ='output'))
  
    opt = Adam(lr= 1e-3/100, decay= 1e-3 /100)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

In [ ]:
model=createmodel()


In [ ]:
history=model.fit_generator(
        train_generator,
        epochs=70,
        validation_data=valid_generator,
        validation_steps=21
        )

In [ ]:
model.save('ksavir.hdf5')

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('./ksavir.hdf5')

prediction=model.predict(test_generator,verbose=1)
print(prediction)

In [ ]:
yp=np.argmax(prediction,axis=1)
print(yp)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,matthews_corrcoef

print('Classification Report')

print(classification_report(test_generator.classes, yp, target_names=directory,digits=4))
print(matthews_corrcoef(test_generator.classes,yp))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()